In [ ]:
from transformers import pipeline, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline
from time import time

## Emotions: https://huggingface.co/SamLowe/roberta-base-go_emotions
## 28 ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 
## 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 
## 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']

## Intents: https://huggingface.co/bespin-global/klue-roberta-small-3i4k-intent-classification?text=What+a+nice+day%21
## 0: "fragment",
## 1: "statement",
## 2: "question",
## 3: "command",
## 4: "rhetorical question",
## 5: "rhetorical command",
## 6: "intonation-dependent utterance"

## Sentiment: https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest
## positive
## neutral
## negative

MODEL_EMOTIONS = 'SamLowe/roberta-base-go_emotions'
MODEL_INTENTS = 'bespin-global/klue-roberta-small-3i4k-intent-classification'
MODEL_SENTIMENT = 'cardiffnlp/twitter-roberta-base-sentiment-latest'


class Text2Vec4TTS:
    
    def __init__(self):

        self.clf_emotion = pipeline(
            task='text-classification', 
            model=MODEL_EMOTIONS, 
            top_k=None
        )

        self.clf_intent = TextClassificationPipeline(
            tokenizer=RobertaTokenizerFast.from_pretrained(MODEL_INTENTS), 
            model=RobertaForSequenceClassification.from_pretrained(MODEL_INTENTS), 
            return_all_scores=True
        )
        
        self.clf_sentiment = pipeline(
            task='sentiment-analysis', 
            model=MODEL_SENTIMENT, 
            tokenizer=MODEL_SENTIMENT, 
            return_all_scores=True
        )
        
    def to_vec(
        self,
        text,
        intent=True,
        emotion=True,
        sentiment=True
    ):
        vec = []
        info = {
            'intent': {},
            'emotion': {},
            'sentiment': {}
        }
        t0 = time()
        
        if intent:
            preds_intent = self.clf_intent(text)[0]
            vec.extend([inta['score'] for inta in sorted(preds_intent, key=lambda x:x['label'])])
            
            for inta in sorted(preds_intent, key=lambda x:-x['score'])[:3]:
                info['intent'].update({inta['label']:inta['score']})
                
        if emotion:
            preds_emotion = self.clf_emotion(text)[0]
            vec.extend([em['score'] for em in sorted(preds_emotion, key=lambda x:x['label'])])
            
            for em in sorted(preds_emotion, key=lambda x:-x['score'])[:3]:
                info['emotion'].update({em['label']:em['score']})
                
        if sentiment:
            preds_sentiment = self.clf_sentiment(text)[0]
            vec.extend([sent['score'] for sent in sorted(preds_sentiment, key=lambda x:x['label'])])
            
            for sent in sorted(preds_sentiment, key=lambda x:-x['score'])[:3]:
                info['sentiment'].update({sent['label']:sent['score']})
            
        info.update({'t': time()-t0})
        return vec, info
    
model = Text2Vec4TTS()

In [ ]:
for text in (
    'What the hell! I hate those disgusting people.',
    'What a beautiful day!'
):
    print(f'\n:: {text}')
    vec, info = model.to_vec(
        text, 
        intent=True, 
        emotion=True, 
        sentiment=True
    )

    print(f'Vector length: {len(vec)}')
    for k, v in info.items():
        if k == 't':
            print(f'\t{k}: {round(v, 2)}s')
        else:
            print(f'\t{k}')
            for kk, sc in v.items():
                print(f'\t\t{kk}: {round(sc, 4)}')